In [1]:
#libraries 
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
             

In [3]:
print(device)

cpu


In [4]:
#data processing
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  #augmentation techniques to artifically increase the sample size\
    transforms.ToTensor(), #PIL image to tensor image with normal pixel range of 0-225 to 0-1
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5]) #changes the range from 0-1 to -1 to 1   
                                                      #formula: (x-mean)/standard deviation
    
])


In [5]:
#loading data (called dataloaders)

#paths for training and testing
train_path = 'C:\\Users\\hqi\\Documents\\seperated data\\training data'
test_path = 'C:\\Users\\hqi\\Documents\\seperated data\\test data'
#dataloader for training data set
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size = 32, shuffle = True #shuffle to make sure the model is not biased towards certain categories
    #batch size used to be 256
)

#dataloader for testing data set
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    batch_size = 16, shuffle = True #shuffle to make sure the model is not biased towards certain categories
)

In [6]:
#fetch all of the classes 
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['benign', 'malignant - melanoma']


In [8]:
#Convolutional neural network class

class ConvNet(nn.Module):
    
    
    #specify all of the layers in the network in the constructor
    def __init__(self, num_classes = 6):
        super(ConvNet,self).__init__()
        
        #input shape: (128, 3, 300, 300)
        
        #apply filter
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        
        #output size after convolutional filter is applied is represented by this equation:
        # ((w-f+2P)/s) +1
        #Current Shape: (128, 12, 300, 300)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1=nn.LeakyReLU()
        #Current Shape: (128, 12, 300, 300)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #reduces output height and width by a factor of 2
        #Current Shape: (128, 12, 150, 150)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 20, 150, 150)
        self.relu2=nn.ReLU()
        #Current Shape: (128, 20, 150, 150)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 32, 150, 150)
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        self.relu3=nn.LeakyReLU()
        #Current Shape: (128, 32, 150, 150)
        
        
        self.conv4=nn.Conv2d(in_channels=32,out_channels=40,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 40, 150, 150)
        self.bn4 = nn.BatchNorm2d(num_features = 40)
        self.relu4=nn.LeakyReLU()
        
        self.conv5=nn.Conv2d(in_channels=40,out_channels=52,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 40, 150, 150)
        self.relu5=nn.ReLU()
        
        self.conv6=nn.Conv2d(in_channels=52,out_channels=64,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 40, 150, 150)
        self.relu6=nn.ReLU()
        
        self.conv7=nn.Conv2d(in_channels=64,out_channels=72,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 40, 150, 150)
        self.relu7=nn.LeakyReLU()
        self.bn7 = nn.BatchNorm2d(num_features = 72)
        
        self.conv8=nn.Conv2d(in_channels=72,out_channels=84,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 40, 150, 150)
        self.relu8=nn.ReLU()


        
        
        self.fc=nn.Linear(in_features=75 * 75 * 84,out_features=num_classes)
       
    
    
    #Feedforward function:
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        output=self.conv4(output)
        output=self.bn4(output)
        output=self.relu4(output)
        
        output=self.conv5(output)
        output=self.relu5(output)
        
        output=self.conv6(output)
        output=self.relu6(output)
        
        output=self.conv7(output)
        output=self.relu7(output)
        output=self.bn7(output)
        
        output=self.conv8(output)
        output=self.relu8(output)
        
        
        

        
        #shape: (256, 52, 75, 75)
        #change shape of matrix to feed into the fc layer
        
        output = output.view(-1, 84*75*75)
        output =self.fc(output)
        return output
        
        
        

In [9]:
model = ConvNet(num_classes = 6).to(device)

In [10]:
#Optimizer
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()


In [11]:
num_epochs = 50

In [12]:
#Calc size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [13]:
print(train_count,test_count)

900 379


In [14]:
#Training model and saving the most accurate model
#Save the model for the epoch that gives the best testing accuracy

best_accuracy = 0

for epoch in range(num_epochs):
    
    #Training on training data set
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #increment for train_loss
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        #increment for train_accuracy
        train_accuracy+=int(torch.sum(prediction==labels.data))
    
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
        
    
    #Evaluating using testing data set
    model.eval()
    test_accuracy = 0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        outputs = model(images)
        _,prediction=torch.max(outputs.data,1)
        
        #increment for train_accuracy
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy = test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    
    
    
    #save best model
    
    if test_accuracy > best_accuracy:
        #save the best model
        torch.save(model.state_dict(),'best2_checkpoint.model')
        #update the value of best_accuracy
        best_accuracy = test_accuracy
    
    

C:\Users\hqi\.conda\envs\pytorch_env\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 Train Loss: tensor(1.4516) Train Accuracy: 0.6877777777777778 Test Accuracy: 0.7968337730870713
Epoch: 1 Train Loss: tensor(0.4579) Train Accuracy: 0.8044444444444444 Test Accuracy: 0.8126649076517151
Epoch: 2 Train Loss: tensor(0.4669) Train Accuracy: 0.8 Test Accuracy: 0.8021108179419525
Epoch: 3 Train Loss: tensor(0.4526) Train Accuracy: 0.82 Test Accuracy: 0.8047493403693932
Epoch: 4 Train Loss: tensor(0.4033) Train Accuracy: 0.8388888888888889 Test Accuracy: 0.8047493403693932
Epoch: 5 Train Loss: tensor(0.3884) Train Accuracy: 0.8311111111111111 Test Accuracy: 0.7757255936675461
Epoch: 6 Train Loss: tensor(0.3798) Train Accuracy: 0.8466666666666667 Test Accuracy: 0.6596306068601583
Epoch: 7 Train Loss: tensor(0.4119) Train Accuracy: 0.8311111111111111 Test Accuracy: 0.7994722955145118
Epoch: 8 Train Loss: tensor(0.4372) Train Accuracy: 0.8277777777777777 Test Accuracy: 0.7994722955145118
Epoch: 9 Train Loss: tensor(0.3191) Train Accuracy: 0.87 Test Accuracy: 0.7651715039